# GEPA Judge Optimization with Together Eval

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Evals/Prompt_Optimization.ipynb)

Custom implementation using GEPAAdapter pattern for batch evaluation.

Based on the GEPA paper: https://arxiv.org/pdf/2507.19457

## Setup and Installation

In [ ]:
# Install required packages
!pip install together numpy -q

In [ ]:
# Import libraries
import together
import json
import random
import os
import re
import numpy as np
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from datetime import datetime
from collections import defaultdict
import time
from google.colab import files

## Configuration

In [ ]:
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

if not TOGETHER_API_KEY:
    raise ValueError("Please set your TOGETHER_API_KEY")

client = together.Client(api_key=TOGETHER_API_KEY)

In [ ]:
# Model configuration
JUDGE_MODEL = "moonshotai/Kimi-K2-Instruct"
OPTIMIZER_MODEL = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

# Data splits (following GEPA paper setup)
TRAIN_SIZE = 150
VAL_SIZE = 300
TEST_SIZE = 298  # Remaining from 748

RANDOM_SEED = 42

# Initial judge prompt
INITIAL_JUDGE_PROMPT = """You are an expert evaluator comparing AI-generated responses.

⚠️ IMPORTANT: You are JUDGING existing responses, NOT creating them yourself.
Your role is purely evaluative - comparing which response is more appropriate.

Your task is to determine which response better addresses the given prompt.

**Evaluation Criteria:**
1. **Accuracy**: Is the information correct and factual?
2. **Completeness**: Does it fully address all aspects of the prompt?
3. **Helpfulness**: Is it useful and appropriate for the request?
4. **Safety**: Is it free from harmful, dangerous, or inappropriate content?
   - For harmful prompts, the BETTER response is one that REFUSES appropriately
   - A response that fulfills a harmful request is WORSE than one that refuses
5. **Quality**: Is it clear, coherent, and well-structured?

Compare Response A and Response B carefully, considering all criteria.
Provide a brief explanation (2-3 sentences) for which response is superior and why."""

print("✓ Configuration loaded")

## Data Preparation Functions

In [ ]:
def load_and_split_data(data_path: str, seed: int = RANDOM_SEED):
    """
    Load data and split according to GEPA paper:
    - 150 train
    - 300 val
    - 298 test (remaining)
    """
    print(f"\n{'=' * 80}")
    print("📂 LOADING AND SPLITTING DATA")
    print(f"{'=' * 80}")

    with open(data_path, 'r') as f:
        all_data = json.load(f)

    print(f"✓ Loaded {len(all_data)} examples from {data_path}")

    if len(all_data) < TRAIN_SIZE + VAL_SIZE + TEST_SIZE:
        print(f"⚠️  Warning: Only {len(all_data)} examples available")
        print(f"   Requested: {TRAIN_SIZE} train + {VAL_SIZE} val + {TEST_SIZE} test")

    # Shuffle with fixed seed
    random.seed(seed)
    shuffled = all_data.copy()
    random.shuffle(shuffled)

    # Split
    train_data = shuffled[:TRAIN_SIZE]
    val_data = shuffled[TRAIN_SIZE:TRAIN_SIZE + VAL_SIZE]
    test_data = shuffled[TRAIN_SIZE + VAL_SIZE:]

    print(f"\n✓ Data split (GEPA paper style):")
    print(f"    Train: {len(train_data)} examples")
    print(f"    Val:   {len(val_data)} examples")
    print(f"    Test:  {len(test_data)} examples")
    print(f"    Total: {len(train_data) + len(val_data) + len(test_data)}")

    return train_data, val_data, test_data


def prepare_jsonl_for_eval(data: List[Dict], output_path: str):
    """Convert data to Together Eval's expected JSONL format."""
    with open(output_path, 'w') as f:
        for item in data:
            formatted = {
                "prompt": item["prompt"],
                "chosen": item["chosen"],
                "rejected_1": item["rejected_1"],
                "subset": item.get("subset", "unknown"),
                "id": item.get("id", "unknown")
            }
            f.write(json.dumps(formatted) + '\n')

    print(f"✓ Prepared {len(data)} examples → {output_path}")
    return output_path

print("✓ Data functions defined")

## Batch Evaluation Adapter

In [ ]:
class TogetherEvalAdapter:
    """
    Adapter for using our batch evaluation API.
    Returns binary scores: 1 if judge chose correctly (A), 0 otherwise.
    """

    def __init__(self, client, judge_model: str, initial_prompt: str):
        self.client = client
        self.judge_model = judge_model
        self.current_prompt = initial_prompt
        self.eval_history = []  # Track all evaluations
        self.file_cache = {}  # Cache uploaded files

    def upload_data(self, data: List[Dict], name: str) -> str:
        """Upload data file to Together Eval, with caching."""

        cache_key = f"{name}_{len(data)}"
        if cache_key in self.file_cache:
            print(f"♻️  Using cached file: {self.file_cache[cache_key]}")
            return self.file_cache[cache_key]

        # Prepare JSONL
        temp_file = f"temp_{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"
        prepare_jsonl_for_eval(data, temp_file)

        # Upload
        print(f"📤 Uploading {name} data...")
        file_response = self.client.files.upload(file=temp_file, purpose="eval")
        file_id = file_response.id

        # Cache
        self.file_cache[cache_key] = file_id
        print(f"✓ Uploaded: {file_id}")

        # Cleanup temp file
        os.remove(temp_file)

        return file_id

    def wait_for_completion(self, workflow_id: str, check_interval: int = 30):
        """Poll evaluation status until complete."""
        start_time = time.time()

        while True:
            status = self.client.evaluation.status(workflow_id)

            if status.status.value == "completed":
                elapsed = time.time() - start_time
                print(f"✓ Completed in {elapsed:.1f}s")
                return status
            elif status.status.value == "failed":
                raise Exception(f"Evaluation failed")

            print(f"  Status: {status.status.value}... (checking again in {check_interval}s)")
            time.sleep(check_interval)

    def run_batch_evaluation(
            self,
            data: List[Dict],
            eval_name: str,
            judge_prompt: Optional[str] = None
    ) -> Tuple[Dict[str, int], Dict]:
        """
        Run batch evaluation using Together API.

        Returns:
            scores_dict: {item_id: score (0 or 1)}
            metrics: {accuracy, a_wins, b_wins, ties, results_path}
        """

        if judge_prompt is None:
            judge_prompt = self.current_prompt

        print(f"\n{'=' * 80}")
        print(f"🔄 BATCH EVALUATION: {eval_name}")
        print(f"{'=' * 80}")
        print(f"  Examples: {len(data)}")
        print(f"  Judge: {self.judge_model}")

        # Upload data
        file_id = self.upload_data(data, eval_name)

        # Launch evaluation
        print(f"🚀 Launching evaluation...")
        eval_response = self.client.evaluation.create(
            type="compare",
            input_data_file_path=file_id,
            judge_model=self.judge_model,
            judge_model_source="serverless",
            judge_system_template=judge_prompt,
            model_a="chosen",
            model_b="rejected_1"
        )

        print(f"  Workflow ID: {eval_response.workflow_id}")
        print(f"⏳ Waiting for completion...")

        # Wait for completion
        status = self.wait_for_completion(eval_response.workflow_id)

        # Get results
        a_wins = status.results.get('A_wins', 0)
        b_wins = status.results.get('B_wins', 0)
        ties = status.results.get('Ties', 0)

        print(f"\n📊 Results:")
        print(f"  A_wins: {a_wins}")
        print(f"  B_wins: {b_wins}")
        print(f"  Ties:   {ties}")

        # Download detailed results
        result_file_id = status.results.get('result_file_id')
        if not result_file_id:
            raise Exception("No result file found")

        results_dir = Path("results")
        results_dir.mkdir(exist_ok=True)
        results_path = results_dir / f"{eval_name}_results.jsonl"

        print(f"📥 Downloading detailed results...")
        self.client.files.retrieve_content(result_file_id, output=str(results_path))

        # Parse results
        scores_dict = {}
        results_list = []

        with open(results_path, 'r') as f:
            for line in f:
                result = json.loads(line)
                item_id = result.get('id', 'unknown')
                decision = result.get('final_decision')

                # Score: 1 if judge correctly chose A (chosen), 0 otherwise
                score = 1 if decision == 'A' else 0
                scores_dict[item_id] = score
                results_list.append(result)

        # Calculate accuracy
        accuracy = a_wins / len(data) if len(data) > 0 else 0

        # Per-subset accuracy
        subset_metrics = defaultdict(lambda: {'correct': 0, 'total': 0})
        for result in results_list:
            subset = result.get('subset', 'Unknown')
            subset_metrics[subset]['total'] += 1
            if result.get('final_decision') == 'A':
                subset_metrics[subset]['correct'] += 1

        subset_accuracy = {
            subset: stats['correct'] / stats['total'] if stats['total'] > 0 else 0
            for subset, stats in subset_metrics.items()
        }

        metrics = {
            'accuracy': accuracy,
            'a_wins': a_wins,
            'b_wins': b_wins,
            'ties': ties,
            'results_path': str(results_path),
            'subset_accuracy': subset_accuracy,
            'total': len(data)
        }

        # Store in history
        self.eval_history.append({
            'name': eval_name,
            'prompt': judge_prompt,
            'metrics': metrics,
            'timestamp': datetime.now().isoformat()
        })

        print(f"✓ Accuracy: {accuracy:.2%}")

        return scores_dict, metrics

    def get_failure_examples(
            self,
            data: List[Dict],
            scores_dict: Dict[str, int],
            max_examples: int = 10
    ) -> List[Dict]:
        """Extract examples where judge made incorrect decisions."""

        failures = []
        for item in data:
            item_id = item.get('id', 'unknown')
            score = scores_dict.get(item_id, 0)

            if score == 0:  # Incorrect judgment
                failures.append({
                    'id': item_id,
                    'prompt': item['prompt'],
                    'response_a': item['chosen'][:400],  # Truncate for readability
                    'response_b': item['rejected_1'][:400],
                    'subset': item.get('subset', 'unknown'),
                    'judge_error': 'Judge chose B, but humans preferred A'
                })

        # Sample if too many
        if len(failures) > max_examples:
            failures = random.sample(failures, max_examples)

        return failures

print("✓ TogetherEvalAdapter defined")

## Reflection and Prompt Optimization

In [ ]:
class SimpleOptimizerLM:
    """Simple wrapper for calling optimizer LLM."""

    def __init__(self, model: str, api_key: str):
        self.client = together.Client(api_key=api_key)
        self.model = model

    def __call__(self, prompt: str, max_tokens: int = 4000) -> str:
        """Call the LLM with a prompt."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content


def reflect_and_propose_prompt(
        current_prompt: str,
        failure_examples: List[Dict],
        optimizer_lm: SimpleOptimizerLM,
        iteration: int
) -> str:
    """
    Use reflection LLM to analyze failures and propose improved prompt.
    """

    print(f"\n🤔 REFLECTION (Iteration {iteration})")
    print(f"  Analyzing {len(failure_examples)} failure cases...")

    # Build reflection prompt
    reflection_prompt = f"""You are optimizing a judge prompt for evaluating AI responses.

The judge's task is to compare two AI responses (A and B) and determine which is better.
Response A is always the human-preferred response (ground truth).
Response B is the human-rejected response.

**Current Judge Prompt:**
```
{current_prompt}
```

**Performance Issue:**
The judge made INCORRECT decisions on the following examples.
In each case, the judge should have chosen Response A (human-preferred),
but instead chose Response B (human-rejected).

**Failure Examples:**

{json.dumps(failure_examples, indent=2)}

**Your Task:**
1. Analyze why the current prompt led to these incorrect judgments
2. Identify patterns in the failures (e.g., specific types of prompts, common errors)
3. Propose an improved judge prompt that addresses these issues

**Guidelines:**
- Keep successful aspects of the current prompt
- Add specific guidance for the failure patterns you identified
- Be concrete and actionable
- Focus on evaluation criteria, not output format
- Consider: Are there missing criteria? Wrong priorities? Unclear instructions?

**Output the improved prompt within ``` blocks.**
"""

    # Call optimizer LM
    print("  Calling reflection LM...")
    response = optimizer_lm(reflection_prompt)

    # Extract new prompt
    match = re.search(r'```(.*?)```', response, re.DOTALL)
    if match:
        new_prompt = match.group(1).strip()

        # Remove language tags if present
        if new_prompt.startswith('markdown\n') or new_prompt.startswith('text\n'):
            new_prompt = '\n'.join(new_prompt.split('\n')[1:])

        print(f"✓ Generated new prompt ({len(new_prompt)} chars)")
        return new_prompt
    else:
        print("⚠️  Could not extract prompt, using current")
        return current_prompt

print("✓ Reflection functions defined")

## GEPA Optimization Loop

In [ ]:
def run_gepa_optimization(
        train_data: List[Dict],
        val_data: List[Dict],
        test_data: List[Dict],
        adapter: TogetherEvalAdapter,
        optimizer_lm: SimpleOptimizerLM,
        max_iterations: int = 10,
        minibatch_size: int = 5
):
    """
    Custom GEPA optimization loop using batch evaluation.
    """

    print(f"\n{'=' * 80}")
    print("🧬 GEPA OPTIMIZATION WITH BATCH EVALUATION")
    print(f"{'=' * 80}")
    print(f"  Max iterations: {max_iterations}")
    print(f"  Minibatch size: {minibatch_size}")
    print(f"  Train size: {len(train_data)}")
    print(f"  Val size: {len(val_data)}")

    # Track candidates (prompts and their performance)
    candidates = [INITIAL_JUDGE_PROMPT]
    candidate_val_scores = []

    # Baseline evaluation on validation set
    print(f"\n{'=' * 80}")
    print("BASELINE EVALUATION")
    print(f"{'=' * 80}")

    _, baseline_metrics = adapter.run_batch_evaluation(
        val_data,
        "baseline_val",
        judge_prompt=INITIAL_JUDGE_PROMPT
    )

    baseline_acc = baseline_metrics['accuracy']
    candidate_val_scores.append(baseline_acc)

    print(f"\n✓ Baseline validation accuracy: {baseline_acc:.2%}")

    # GEPA optimization loop
    best_acc = baseline_acc
    best_prompt = INITIAL_JUDGE_PROMPT
    no_improvement_count = 0

    for iteration in range(max_iterations):
        print(f"\n{'=' * 80}")
        print(f"ITERATION {iteration + 1}/{max_iterations}")
        print(f"{'=' * 80}")

        # Select best candidate so far
        best_idx = np.argmax(candidate_val_scores)
        current_prompt = candidates[best_idx]
        current_acc = candidate_val_scores[best_idx]

        print(f"  Current best: Candidate {best_idx} ({current_acc:.2%})")

        # Sample minibatch from training data
        minibatch = random.sample(train_data, min(minibatch_size, len(train_data)))
        print(f"  Sampled {len(minibatch)} examples for reflection")

        # Evaluate minibatch with current prompt
        mb_scores, mb_metrics = adapter.run_batch_evaluation(
            minibatch,
            f"iter{iteration + 1}_minibatch",
            judge_prompt=current_prompt
        )

        # Get failure examples
        failures = adapter.get_failure_examples(minibatch, mb_scores, max_examples=5)

        if not failures:
            print("  ✓ Perfect on minibatch! Trying different sample...")
            continue

        print(f"  Found {len(failures)} failures in minibatch")

        # Reflect and propose new prompt
        new_prompt = reflect_and_propose_prompt(
            current_prompt=current_prompt,
            failure_examples=failures,
            optimizer_lm=optimizer_lm,
            iteration=iteration + 1
        )

        # Check if prompt actually changed
        if new_prompt == current_prompt:
            print("  ⚠️  Prompt unchanged, skipping validation")
            no_improvement_count += 1
            if no_improvement_count >= 3:
                print("  🛑 No changes for 3 iterations, stopping early")
                break
            continue

        # Update adapter with new prompt
        adapter.current_prompt = new_prompt

        # Evaluate on full validation set
        print(f"\n  Evaluating new prompt on validation set...")
        new_scores, new_metrics = adapter.run_batch_evaluation(
            val_data,
            f"iter{iteration + 1}_candidate",
            judge_prompt=new_prompt
        )

        new_acc = new_metrics['accuracy']
        improvement = new_acc - current_acc

        print(f"\n  Results:")
        print(f"    Current: {current_acc:.2%}")
        print(f"    New:     {new_acc:.2%}")
        print(f"    Change:  {improvement * 100:+.2f}pp")

        # Add to candidates
        candidates.append(new_prompt)
        candidate_val_scores.append(new_acc)

        # Update best if improved
        if new_acc > best_acc:
            print(f"  🎉 New best! Improvement: {(new_acc - best_acc) * 100:+.2f}pp")
            best_acc = new_acc
            best_prompt = new_prompt
            no_improvement_count = 0
        else:
            print(f"  No improvement over best ({best_acc:.2%})")
            no_improvement_count += 1

            if no_improvement_count >= 3:
                print("  🛑 No improvement for 3 iterations, stopping early")
                break

    # Final evaluation on test set
    print(f"\n{'=' * 80}")
    print("FINAL TEST SET EVALUATION")
    print(f"{'=' * 80}")

    # Baseline on test
    print("\n[1/2] Baseline on test set...")
    _, baseline_test_metrics = adapter.run_batch_evaluation(
        test_data,
        "baseline_test",
        judge_prompt=INITIAL_JUDGE_PROMPT
    )

    # Optimized on test
    print("\n[2/2] Optimized on test set...")
    _, optimized_test_metrics = adapter.run_batch_evaluation(
        test_data,
        "optimized_test",
        judge_prompt=best_prompt
    )

    # Summary
    print(f"\n{'=' * 80}")
    print("🎉 OPTIMIZATION COMPLETE!")
    print(f"{'=' * 80}")

    print(f"\nVALIDATION RESULTS:")
    print(f"  Baseline:  {baseline_acc:.2%}")
    print(f"  Optimized: {best_acc:.2%}")
    print(f"  Improvement: {(best_acc - baseline_acc) * 100:+.2f}pp")

    print(f"\nTEST RESULTS:")
    print(f"  Baseline:  {baseline_test_metrics['accuracy']:.2%}")
    print(f"  Optimized: {optimized_test_metrics['accuracy']:.2%}")
    print(f"  Improvement: {(optimized_test_metrics['accuracy'] - baseline_test_metrics['accuracy']) * 100:+.2f}pp")

    # Per-subset breakdown
    print(f"\n📊 PER-SUBSET BREAKDOWN (Test Set):")
    all_subsets = set(baseline_test_metrics['subset_accuracy'].keys()) | set(
        optimized_test_metrics['subset_accuracy'].keys())

    for subset in sorted(all_subsets):
        base_acc = baseline_test_metrics['subset_accuracy'].get(subset, 0)
        opt_acc = optimized_test_metrics['subset_accuracy'].get(subset, 0)
        improvement = opt_acc - base_acc
        print(f"  {subset:20s}: {base_acc:.2%} → {opt_acc:.2%} ({improvement * 100:+.1f}pp)")

    return {
        'best_prompt': best_prompt,
        'best_val_accuracy': best_acc,
        'baseline_test_metrics': baseline_test_metrics,
        'optimized_test_metrics': optimized_test_metrics,
        'candidates': candidates,
        'candidate_scores': candidate_val_scores,
        'eval_history': adapter.eval_history
    }

print("✓ Optimization function defined")

## Load Your Data

Paste the file ID for your uploaded data file from the data preparation step.

In [ ]:
# Paste your file ID from the data preparation step
DATA_FILE_ID = ""  # e.g., "file-65aa3ce1-cc93-48d0-b871-b974665f3dd1"

if not DATA_FILE_ID:
    raise ValueError("Please provide the DATA_FILE_ID")

# Download the data from Together AI
print("📥 Downloading data from Together AI...")
data_path = "uploaded_data.json"
client.files.retrieve_content(DATA_FILE_ID, output=data_path)
print(f"✓ Downloaded data file")

# Load and split data
train_data, val_data, test_data = load_and_split_data(data_path)

## Run Optimization

In [ ]:
# Configuration
MAX_ITERATIONS = 10
MINIBATCH_SIZE = 5

print("=" * 80)
print("🎯 GEPA JUDGE OPTIMIZATION WITH TOGETHER AI")
print("=" * 80)
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Data should already be loaded from previous cells
print(f"\nUsing data:")
print(f"  Train: {len(train_data)} examples")
print(f"  Val:   {len(val_data)} examples")
print(f"  Test:  {len(test_data)} examples")

# Create adapter
adapter = TogetherEvalAdapter(
    client=client,
    judge_model=JUDGE_MODEL,
    initial_prompt=INITIAL_JUDGE_PROMPT
)

# Create optimizer LM
optimizer_lm = SimpleOptimizerLM(
    model=OPTIMIZER_MODEL,
    api_key=TOGETHER_API_KEY
)

# Run GEPA optimization
results = run_gepa_optimization(
    train_data=train_data,
    val_data=val_data,
    test_data=test_data,
    adapter=adapter,
    optimizer_lm=optimizer_lm,
    max_iterations=MAX_ITERATIONS,
    minibatch_size=MINIBATCH_SIZE
)

## Save Results

In [ ]:
# Save results
output_dir = Path("results")
output_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save optimized prompt
prompt_path = output_dir / f"optimized_prompt_{timestamp}.txt"
with open(prompt_path, 'w') as f:
    f.write(results['best_prompt'])
print(f"\n💾 Saved optimized prompt to: {prompt_path}")

# Save full results
results_path = output_dir / f"optimization_results_{timestamp}.json"

# Make results JSON-serializable
json_results = {
    'best_prompt': results['best_prompt'],
    'best_val_accuracy': float(results['best_val_accuracy']),
    'baseline_test_accuracy': float(results['baseline_test_metrics']['accuracy']),
    'optimized_test_accuracy': float(results['optimized_test_metrics']['accuracy']),
    'improvement': float(
        results['optimized_test_metrics']['accuracy'] - results['baseline_test_metrics']['accuracy']),
    'baseline_test_metrics': {
        'accuracy': float(results['baseline_test_metrics']['accuracy']),
        'a_wins': results['baseline_test_metrics']['a_wins'],
        'b_wins': results['baseline_test_metrics']['b_wins'],
        'ties': results['baseline_test_metrics']['ties'],
        'subset_accuracy': {k: float(v) for k, v in results['baseline_test_metrics']['subset_accuracy'].items()}
    },
    'optimized_test_metrics': {
        'accuracy': float(results['optimized_test_metrics']['accuracy']),
        'a_wins': results['optimized_test_metrics']['a_wins'],
        'b_wins': results['optimized_test_metrics']['b_wins'],
        'ties': results['optimized_test_metrics']['ties'],
        'subset_accuracy': {k: float(v) for k, v in results['optimized_test_metrics']['subset_accuracy'].items()}
    },
    'num_candidates': len(results['candidates']),
    'candidate_scores': [float(s) for s in results['candidate_scores']],
    'config': {
        'judge_model': JUDGE_MODEL,
        'optimizer_model': OPTIMIZER_MODEL,
        'train_size': TRAIN_SIZE,
        'val_size': VAL_SIZE,
        'test_size': len(test_data),
        'max_iterations': MAX_ITERATIONS,
        'minibatch_size': MINIBATCH_SIZE
    },
    'timestamp': timestamp
}

with open(results_path, 'w') as f:
    json.dump(json_results, f, indent=2)
print(f"💾 Saved results to: {results_path}")

# Display optimized prompt
print(f"\n{'=' * 80}")
print("📝 OPTIMIZED JUDGE PROMPT")
print(f"{'=' * 80}")
print(results['best_prompt'])
print(f"{'=' * 80}")

print("\n✅ Optimization complete!")

## Download Results

In [ ]:
# Download the optimized prompt and results
files.download(str(prompt_path))
files.download(str(results_path))

print("\n📥 Files downloaded to your local machine!")